In [ ]:
# Adding local Keras
import sys
import os
import numpy as np
from skimage import io
import itertools
from IPython.core.debugger import Tracer
%matplotlib inline
import matplotlib.pyplot as plt
KERAS_PATH = '/media/manish/Data/keras-master/'
sys.path.insert(0, KERAS_PATH)
sys.path.insert(0, os.path.join(KERAS_PATH, 'keras'))
sys.path.insert(0, os.path.join(KERAS_PATH, 'keras', 'layers'))
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
"""
data_weighted_loader consider the label for specific treatments
"""
def data_weighted_loader(path, batch_size, ignore_val=44, pos_val=255, neg_val=155, pos_class=3, neg_class=4):
    # pos_class and neg_class in the folder name for keras ImageDataGenerator input
    # 0,1,2 is for previous data. 3 and 4 is for new tcga data, which needs specific treatment

    def imerge(img_gen, mask_gen):
        
        for (img, img_labels), (mask, mask_labels) in itertools.izip(img_gen, mask_gen):
            Tracer()()
            
            # weight
            mask = np.expand_dims(mask[:,:,:,0], axis=3)
#             for img in mask:
#                 plt.figure()
#                 plt.imshow(img)
            print("Shape of the mask_labels: %s" % str(mask_labels))
            print("Shape of the label mask: %s" % str(mask.shape))
            
            weight = np.ones(mask.shape, np.float)
            weight[mask==ignore_val] = 0
            # In mask, ignored pixel has value ignore_val.
            # The weight of these pixel is set to zero, so they do not contribute to loss
            # The returned mask is still binary.

            # compute per sample
            for c, mask_label in enumerate(mask_labels):
                assert(mask_labels[c] == img_labels[c])
                mask_sample = mask[c]
                if mask_label == pos_class:
                    assert(np.where(mask_sample == neg_val)[0].size == 0)
                    mask_sample[mask_sample==pos_val] = 1
                elif mask_label == neg_class:
                    assert(np.where(mask_sample == pos_val)[0].size == 0)
                    mask_sample[mask_sample==neg_val] = 0
                else:
                	mask_sample /= 255

                mask_sample[mask_sample==ignore_val] = 0

            assert set(np.unique(mask)).issubset([0, 1])
            assert set(np.unique(weight)).issubset([0, 1])

            yield img/255.0, mask, weight, img_labels
            # yield norm_mean_img(img), mask, weight, img_labels

    train_data_gen_args = dict(
                    horizontal_flip=True,
                    vertical_flip=True,
                    zoom_range=0.1,
                    fill_mode='reflect')

    seed = 1234
    train_image_datagen = ImageDataGenerator(**train_data_gen_args).flow_from_directory(
                                path+'train/img',
                                class_mode="sparse",
                                target_size=(512, 512),
                                batch_size=batch_size,
                                seed=seed)
    train_mask_datagen = ImageDataGenerator(**train_data_gen_args).flow_from_directory(
                                path+'train/groundtruth',
                                class_mode="sparse",
                                target_size=(512, 512),
                                batch_size=batch_size,
                                seed=seed)

    test_image_datagen = ImageDataGenerator().flow_from_directory(
                                path+'val/img',
                                class_mode="sparse",
                                target_size=(512, 512),
                                batch_size=batch_size,
                                seed=seed)
    test_mask_datagen = ImageDataGenerator().flow_from_directory(
                                path+'val/groundtruth',
                                class_mode="sparse",
                                target_size=(512, 512),
                                batch_size=batch_size,
                                seed=seed)

    train_generator = imerge(train_image_datagen, train_mask_datagen)
    test_generator = imerge(test_image_datagen, test_mask_datagen)

    return train_generator, test_generator

training_dir = './data/BladderData/Segmentation/'
batch_size = 5
train_generator, _ = data_weighted_loader(training_dir, batch_size)
x_batch, y_batch, weight, _ = train_generator.next()h

In [ ]:
#--- Code taken from keras ---#
import os
def list_valid_filenames_in_directory(directory, white_list_formats,
                                       class_indices, follow_links=False):
    """List paths of files in `subdir` relative from `directory` whose extensions are in `white_list_formats`.
    # Arguments
    directory: absolute path to a directory containing the files to list.
    The directory name is used as class label and must be a key of `class_indices`.
    white_list_formats: set of strings containing allowed extensions for
    the files to be counted.
    class_indices: dictionary mapping a class name to its index.
    # Returns
    classes: a list of class indices
    filenames: the path of valid files in `directory`, relative from
    `directory`'s parent (e.g., if `directory` is "dataset/class1",
    the filenames will be ["class1/file1.jpg", "class1/file2.jpg", ...]).
    """
    def _recursive_list(subpath):
        return sorted(os.walk(subpath, followlinks=follow_links), key=lambda tpl: tpl[0])
    
    classes = []
    filenames = []
    subdir = os.path.basename(directory)
    basedir = os.path.dirname(directory)
    for root, _, files in _recursive_list(directory):
        for fname in files:
            is_valid = False
            for extension in white_list_formats:
                if fname.lower().endswith('.' + extension):
                    is_valid = True
                    break
            if is_valid:
                classes.append(class_indices[subdir])
                # add filename relative to directory
                absolute_path = os.path.join(root, fname)
                filenames.append(os.path.relpath(absolute_path, basedir))
            
    return classes, filenames

directory = '/media/manish/Data/Research/Lab Software Development/'
directory += 'BreastCancerPositive/BreastCancerPositive/static/Annotations'
classes = []
for subdir in sorted(os.listdir(directory)):
    if os.path.isdir(os.path.join(directory, subdir)):
        classes.append(subdir)
        
num_class = len(classes)
class_indices = dict(zip(classes, range(len(classes))))
samples = 0
white_list_formats = {'png', 'jpg', 'jpeg', 'bmp', 'json'}

def _recursive_list(subpath):
    return sorted(os.walk(subpath, followlinks=False), key=lambda tpl: tpl[0])

for subdir in classes:
    subpath = os.path.join(directory, subdir)
    for root, _, files in _recursive_list(subpath):
        for fname in files:
            is_valid = False
            for extension in white_list_formats:
                if fname.lower().endswith('.' + extension):
                    is_valid = True
                    break
            if is_valid:
                samples += 1
    
print('Found %d images belonging to %d classes.' % (samples, num_class))

# second, build an index of the images in the different class subfolders
filenames = []
i = 0
for subdir in classes:
    subpath = os.path.join(directory, subdir)
    _, fnames = list_valid_filenames_in_directory(subpath, white_list_formats,
                                                  class_indices)
    pprint(fnames)
    filenames.append(fnames)

print('Found %d files.' % len(filenames))

In [ ]:
# Read the json file and whole slides and pick random points from annotation to crop the patch
# put positive patch in 1 folder and negative patch in 0 folder
import json
from pprint import pprint
from matplotlib import path as matpath
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from IPython.core.debugger import Tracer
import openslide_util as ou
import numpy as np
from PIL import Image

img_crop_size = 512;
offset_x_left = img_crop_size/2;
offset_y_top = img_crop_size/2;

def crop_img_mask(polygon, 
                  center, offset_x, offset_y, 
                  path=None,
                  img_crop_size=512, whole_slide_size=(90000, 90000)):
    print("Creating offsets")
    startX = int(center[0] - offset_x)
    startY = int(center[1] - offset_y)
    endX = int(startX + img_crop_size)
    endY = int(startY + img_crop_size)
    
    if startX<0:
        startX = 100;
    if startY<0:
        startY = 100;
    if endX > whole_slide_size[0]:
        endX = whole_slide_size[0] - 100
    if endY > whole_slide_size[1]:
        endY = whole_slide_size[1] - 100
    
    print("Getting the mask")
    xv,yv = np.meshgrid(np.arange(startX, endX, 1), np.arange(startY, endY, 1))
    flags = polygon.contains_points(np.hstack((xv.flatten()[:, np.newaxis], yv.flatten()[:, np.newaxis])))
    mask = np.zeros((img_crop_size, img_crop_size), dtype='bool')
    mask[(yv.flatten() - startY).astype('int'), (xv.flatten() - startX).astype('int')] = flags
    
    img =[]
    
    if path is not None:
        print("Reading whole slide section")
        print(startX, startY)
        img, _ = ou.crop_slideimage(path, 
                                    location=(startX, startY),
                                    level=0,
                                    size=(img_crop_size, img_crop_size))
        
    return img, mask

def generate_random(number, polygon):
    list_of_points = []
    bound = polygon.get_extents().get_points().reshape(1,-1)[0].tolist()
    pprint(bound)
    minx, miny, maxx, maxy = bound[0], bound[1], bound[2], bound[3]
    
    counter = 0
    while counter < number:
        pnt = [np.random.uniform(minx, maxx), np.random.uniform(miny, maxy)]
        if polygon.contains_point(pnt):
            list_of_points.append(pnt)
            counter += 1
    return list_of_points

#-----------The main good stuffs---------------#
    
#------------Reading all the images from the directory with sub-directory as class labels----------#
#'/media/manish/Data/Research/Breast Cancer Project/code/data/
#TCGA-3C-AALJ-01Z-00-DX2.62DFE56B-B84C-40F9-9625-FCB55767B70D/annotations.json'

def generate_date_from_whole_slide(image_path, 
                                   annotation_path, 
                                   save_path,
                                   image_identifier,
                                   usage_identifier='train', 
                                   folder_class='positive')
    '''Crops the region annotation data from whole slide and json data and saves for train or validation
    image_path : path for the whole slide
    annotation_path : path for the region annotation json data 
    save_path :  root folder path 
    image_identifier : identifier of the whole slide image, e.g. index of the read whole slide image
    usage_identifier :  will be used for training or validation
    folder_class : positive or negative annotation data
    '''
    print('Reading annotation data...')
    with open(annotation_path) as data_file:    
        data = json.load(data_file)

    #path = '/media/manish/Data/Research/Breast Cancer Project/code/'
    #path = path + 'data/TCGA-3C-AALJ-01Z-00-DX2.62DFE56B-B84C-40F9-9625-FCB55767B70D.svs'

    whole_slide_size = ou.get_slide_dimension(slide_path=image_path)

    pprint(whole_slide_size)
    
    count = 0;
    for index in range(len(data['Regions'])):
        points = data['Regions'][index]['points']
        polygon_points = []

        for i in range(len(points)):
            x = points[i]['x']
            y = points[i]['y']
            polygon_points.append(x)
            polygon_points.append(y)

        polygon_points = np.array(polygon_points)
        polygon_points = np.reshape(polygon_points, (-1, 2))
        p = matpath.Path(polygon_points, _interpolation_steps=1, closed=True)

        # get random points from the polygon
        # [[26861.936345050577, 68854.72624774669]]
        # [[26711.20473907492, 70423.97652441125]]
        num_sample_points = min(100, polygon_points.shape[0])
        random_centers = generate_random(num_sample_points, p)
        pprint(random_centers)

        print(path)
        
        for row in random_centers:
            img, mask = crop_img_mask(p, 
                                 row, offset_x_left, 
                                 offset_y_top, 
                                 path=path,
                                 whole_slide_size=whole_slide_size)
            mask = Image.fromarray(mask.astype('uint8')*255)
            
            img_save_path = os.path.join(save_path, usage_identifier, folder_class)
            mask_save_path = os.path.join(save_path, 'groundtruth', usage_identifier, folder_class)
            
            img.save(os.path.join(img_save_path, 'img_' + image_identifier + '_' + str(count) + '.png'))
            mask.save(os.path.join(mask_save_path, 
                                   'img_mask_' + image_identifier + '_' + str(count) + '.png', ))
            #plt.figure()
            #plt.imshow(mask, cmap='gray', vmin=0, vmax=1)
            count += 1

    print('Saved %d ROIs for %s image' % (count, os.path.basename(image_path)))